**Demo for `teneva.core.als`**

---

This module contains the function `als` which computes the TT-approximation for the tensor by TT-ALS algorithm, using given random samples.

## Loading and importing modules

In [1]:
import numpy as np
import teneva
from time import perf_counter as tpc
np.random.seed(42)

## Function `als`

Build TT-tensor by TT-ALS from the given random tensor samples.

In [2]:
d         = 5                           # Dimension of the function
a         = [-5., -4., -3., -2., -1.]   # Lower bounds for spatial grid
b         = [+6., +3., +3., +1., +2.]   # Upper bounds for spatial grid
n         = [ 20,  18,  16,  14,  12]   # Shape of the tensor

In [3]:
m         = 1.E+4  # Number of calls to target function
nswp      = 50     # Sweep number for ALS iterations
r         = 3      # TT-rank of the initial random tensor

We set the target function (the function takes as input a set of tensor multi-indices I of the shape [samples, dimension], which are transformed into points X of a uniform spatial grid using the function "ind_to_poi"):

In [4]:
from scipy.optimize import rosen
def func(I): 
    X = teneva.ind_to_poi(I, a, b, n)
    return rosen(X.T)

We prepare train data from the LHS random distribution:

In [5]:
I_trn = teneva.sample_lhs(n, m) 
Y_trn = func(I_trn)

We prepare test data from as a random tensor multi-indices:

In [6]:
# Test data:

# Number of test points:
m_tst = int(1.E+4)

# Random multi-indices for the test points:
I_tst = np.vstack([np.random.choice(n[i], m_tst) for i in range(d)]).T

# Function values for the test points:
Y_tst = func(I_tst)

We build the TT-tensor, which approximates the target function (we generate random initial r-rank approximation in the TT-format using the function "rand" and then compute the resulting TT-tensor by TT-ALS):

In [7]:
t = tpc()
Y = teneva.rand(n, r)
Y = teneva.als(I_trn, Y_trn, Y, nswp)
t = tpc() - t

print(f'Build time     : {t:-10.2f}')

Build time     :       1.58


And now we can check the result:

In [8]:
# Fast getter for TT-tensor values:
get = teneva.getter(Y)                     

# Compute approximation in train points:
Z = np.array([get(i) for i in I_trn])

# Accuracy of the result for train points:
e_trn = np.linalg.norm(Z - Y_trn)          
e_trn /= np.linalg.norm(Y_trn)

# Compute approximation in test points:
Z = np.array([get(i) for i in I_tst])

# Accuracy of the result for test points:
e_tst = np.linalg.norm(Z - Y_tst)          
e_tst /= np.linalg.norm(Y_tst)

print(f'Error on train : {e_trn:-10.2e}')
print(f'Error on test  : {e_tst:-10.2e}')

Error on train :   3.23e-15
Error on test  :   3.51e-15


## Function `als2`

Build TT-tensor by TT-ALS from the given random tensor samples. This is the alternative realization of the algorithm. The version from "als" function in many cases works better and much faster.

In [9]:
d         = 5                           # Dimension of the function
a         = [-5., -4., -3., -2., -1.]   # Lower bounds for spatial grid
b         = [+6., +3., +3., +1., +2.]   # Upper bounds for spatial grid
n         = [ 20,  18,  16,  14,  12]   # Shape of the tensor

In [10]:
m         = 1.E+4  # Number of calls to target function
nswp      = 50     # Sweep number for ALS iterations
r         = 3      # TT-rank of the initial random tensor

We set the target function (the function takes as input a set of tensor multi-indices I of the shape [samples, dimension], which are transformed into points X of a uniform spatial grid using the function "ind_to_poi"):

In [11]:
from scipy.optimize import rosen
def func(I): 
    X = teneva.ind_to_poi(I, a, b, n)
    return rosen(X.T)

We prepare train data from the LHS random distribution:

In [12]:
I_trn = teneva.sample_lhs(n, m) 
Y_trn = func(I_trn)

We prepare test data from as a random tensor multi-indices:

In [13]:
# Test data:

# Number of test points:
m_tst = int(1.E+4)

# Random multi-indices for the test points:
I_tst = np.vstack([np.random.choice(n[i], m_tst) for i in range(d)]).T

# Function values for the test points:
Y_tst = func(I_tst)

We build the TT-tensor, which approximates the target function (we generate random initial r-rank approximation in the TT-format using the function "rand" and then compute the resulting TT-tensor by TT-ALS):

In [14]:
t = tpc()
Y = teneva.rand(n, r)
Y = teneva.als2(I_trn, Y_trn, Y, nswp)
t = tpc() - t

print(f'Build time     : {t:-10.2f}')

Build time     :      89.15


And now we can check the result:

In [15]:
# Fast getter for TT-tensor values:
get = teneva.getter(Y)                     

# Compute approximation in train points:
Z = np.array([get(i) for i in I_trn])

# Accuracy of the result for train points:
e_trn = np.linalg.norm(Z - Y_trn)          
e_trn /= np.linalg.norm(Y_trn)

# Compute approximation in test points:
Z = np.array([get(i) for i in I_tst])

# Accuracy of the result for test points:
e_tst = np.linalg.norm(Z - Y_tst)          
e_tst /= np.linalg.norm(Y_tst)

print(f'Error on train : {e_trn:-10.2e}')
print(f'Error on test  : {e_tst:-10.2e}')

Error on train :   1.22e-15
Error on test  :   1.33e-15


---